## Tp2 ift 870
Quentin Levieux<br>
Frederic Giasson

In [39]:
import sklearn
import numpy as np
import scipy as sp
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import numbers

In [40]:
package = pd.read_csv("Package2.csv",sep=";", error_bad_lines=False, encoding ='latin1')
display("Dimension de package.csv : ",package.shape)
product = pd.read_csv("Product2.csv",sep=";", error_bad_lines=False, encoding ='latin1')
print("Dimension de product.csv: ",product.shape)
print("Nombre d'attributs de package:", len(package.columns))
print("Nombre d'attributs de product:", len(product.columns))
print("\nColonne de package: \n",package.columns.values)
print("\nColonne de product: \n",product.columns.values)
commonAttributs = list(set(package).intersection(product))
print("\nAttributs commun au deux tables :\n",commonAttributs)

'Dimension de package.csv : '

(173887, 8)

Dimension de product.csv:  (93238, 20)
Nombre d'attributs de package: 8
Nombre d'attributs de product: 20

Colonne de package: 
 ['PRODUCTID' 'PRODUCTNDC' 'NDCPACKAGECODE' 'PACKAGEDESCRIPTION'
 'STARTMARKETINGDATE' 'ENDMARKETINGDATE' 'NDC_EXCLUDE_FLAG'
 'SAMPLE_PACKAGE']

Colonne de product: 
 ['PRODUCTID' 'PRODUCTNDC' 'PRODUCTTYPENAME' 'PROPRIETARYNAME'
 'PROPRIETARYNAMESUFFIX' 'NONPROPRIETARYNAME' 'DOSAGEFORMNAME' 'ROUTENAME'
 'STARTMARKETINGDATE' 'ENDMARKETINGDATE' 'MARKETINGCATEGORYNAME'
 'APPLICATIONNUMBER' 'LABELERNAME' 'SUBSTANCENAME'
 'ACTIVE_NUMERATOR_STRENGTH' 'ACTIVE_INGRED_UNIT' 'PHARM_CLASSES'
 'DEASCHEDULE' 'NDC_EXCLUDE_FLAG' 'LISTING_RECORD_CERTIFIED_THROUGH']

Attributs commun au deux tables :
 ['STARTMARKETINGDATE', 'NDC_EXCLUDE_FLAG', 'ENDMARKETINGDATE', 'PRODUCTNDC', 'PRODUCTID']


## Types des attributs

In [41]:
print("Types des attributs pour package :\n\n",package.dtypes)
print("\nTypes des attributs pour product :\n\n",product.dtypes)

Types des attributs pour package :

 PRODUCTID              object
PRODUCTNDC             object
NDCPACKAGECODE         object
PACKAGEDESCRIPTION     object
STARTMARKETINGDATE      int64
ENDMARKETINGDATE      float64
NDC_EXCLUDE_FLAG       object
SAMPLE_PACKAGE         object
dtype: object

Types des attributs pour product :

 PRODUCTID                            object
PRODUCTNDC                           object
PRODUCTTYPENAME                      object
PROPRIETARYNAME                      object
PROPRIETARYNAMESUFFIX                object
NONPROPRIETARYNAME                   object
DOSAGEFORMNAME                       object
ROUTENAME                            object
STARTMARKETINGDATE                    int64
ENDMARKETINGDATE                    float64
MARKETINGCATEGORYNAME                object
APPLICATIONNUMBER                    object
LABELERNAME                          object
SUBSTANCENAME                        object
ACTIVE_NUMERATOR_STRENGTH            object
ACTIVE_INGR

### Valeurs Manquantes

In [42]:
manquantePackage = package[package.isnull().any(axis=1)]
#display(manquantePackage)
print(manquantePackage.shape)

manquanteProduct = product[product.isnull().any(axis=1)]
#display(manquanteProduct)
print(manquanteProduct.shape)

print("Valeurs manquante par colonne dans package:\n")
for col in package.columns:
    nb = package[[col]].isna().sum()
    print(nb)
print("Nombre total de rangees dans package :", len(package))

print("\nValeurs manquante par colonne dans product:\n")
for col in product.columns:
    nb = product[[col]].isna().sum()
    print(nb)
print("Nombre total de rangees dans product:", len(product))

(167507, 8)
(93238, 20)
Valeurs manquante par colonne dans package:

PRODUCTID    0
dtype: int64
PRODUCTNDC    1500
dtype: int64
NDCPACKAGECODE    2346
dtype: int64
PACKAGEDESCRIPTION    0
dtype: int64
STARTMARKETINGDATE    0
dtype: int64
ENDMARKETINGDATE    167431
dtype: int64
NDC_EXCLUDE_FLAG    0
dtype: int64
SAMPLE_PACKAGE    0
dtype: int64
Nombre total de rangees dans package : 173887

Valeurs manquante par colonne dans product:

PRODUCTID    1560
dtype: int64
PRODUCTNDC    0
dtype: int64
PRODUCTTYPENAME    0
dtype: int64
PROPRIETARYNAME    6
dtype: int64
PROPRIETARYNAMESUFFIX    83075
dtype: int64
NONPROPRIETARYNAME    4
dtype: int64
DOSAGEFORMNAME    0
dtype: int64
ROUTENAME    1932
dtype: int64
STARTMARKETINGDATE    0
dtype: int64
ENDMARKETINGDATE    88915
dtype: int64
MARKETINGCATEGORYNAME    0
dtype: int64
APPLICATIONNUMBER    13097
dtype: int64
LABELERNAME    0
dtype: int64
SUBSTANCENAME    2309
dtype: int64
ACTIVE_NUMERATOR_STRENGTH    2309
dtype: int64
ACTIVE_INGRED_UNIT  

### incohérences intra-attributs

In [45]:
#Méthode pour déterminer si la date est valide
def datee(date):
    annee = int(str(date)[0:4])
    mois = int(str(date)[4:6])
    jour = int(str(date)[6:8])
    if annee<1800 or annee > 2300 or mois>12 or mois<1:
        return False
    if jour<1 or jour>31:
        return False
    if mois== 2 and jour>29:
        return False
    if mois== 4 and jour>30:
        return False
    if mois== 6 and jour>30:
        return False
    if mois== 9 and jour>30:
        return False
    if mois== 11 and jour>30:
        return False
    return True


In [46]:
#display(package.head())
#incoherence dans la colonne de package
samplePackageY = package[package['SAMPLE_PACKAGE'] != 'Y']
samplePackageN = samplePackageY[samplePackageY['SAMPLE_PACKAGE'] != 'N']
print("Nombre de rangee avec une valeur autre que Y ou N dans la colonne sample_Package : ",len(samplePackageN))

ndc_exclude_flagY = package[package['NDC_EXCLUDE_FLAG'] != 'Y']
ndc_exclude_flagN = ndc_exclude_flagY[ndc_exclude_flagY['NDC_EXCLUDE_FLAG'] != 'N']
print("Nombre de rangee avec une valeur autre que Y ou N (ni E ou I) dans la colonne ndc_exclude_flag : ",len(samplePackageN))

xx = package[~package['ENDMARKETINGDATE'].isnull()]
endMarketingDate = xx['ENDMARKETINGDATE']
erreurEndMarketingDate = []
for date in endMarketingDate:
    if not datee(date):
        erreurEndMarketingDate.append(date)
print("Nombre incoherence dans endMarketingDate: ", len(erreurEndMarketingDate))

yy = package[~package['STARTMARKETINGDATE'].isnull()]
startMarketingDate = yy['STARTMARKETINGDATE']
erreurStartdMarketingDate = []
for date in startMarketingDate:
    if not datee(date):
        erreurStartMarketingDate.append(date)
print("Nombre incoherence dans startMarketingDate: ", len(erreurStartdMarketingDate))

vv = package[~package['PRODUCTNDC'].isnull()]
ndcErr = []
for ndc in vv['PRODUCTNDC'].unique():
    ndc = ndc.strip('-')
    if not any(char.isdigit() for char in ndc):
           ndcErr.append(ndc)
print("Nombres de differentes possibilites pour PRODUCTNDC:",len(ndcErr), ndcErr)

bb = package[~package['PRODUCTID'].isnull()]
print("Nombres de differentes possibilites pour PRODUCTID:",len(bb), len(bb['PRODUCTID'].unique()))
errID = bb[bb['PRODUCTID'].duplicated()== True]
errID = errID['PRODUCTID']
print(len(errID))

Nombre de rangee avec une valeur autre que Y ou N dans la colonne sample_Package :  0
Nombre de rangee avec une valeur autre que Y ou N (ni E ou I) dans la colonne ndc_exclude_flag :  0
Nombre incoherence dans endMarketingDate:  0
Nombre incoherence dans startMarketingDate:  0
Nombres de differentes possibilites pour PRODUCTNDC: 0 []
Nombres de differentes possibilites pour PRODUCTID: 173887 93084
80803


En ce qui attrait au colonnes <b>NDCPACKAGECODE PACKAGEDESCRIPTION</b> de la table PACKAGE, il est difficile de d'avoir un critère qui permet de savoir si il y a une incoherence dans les données de la colonne, le site n'offre pas plus de details. 
<br><br>Dans la colonne <b>PRODUCTID</b> nous pouvons voir qu'il y a 80803 identifiants qui ne sont pas unique, nous pouvons soupconner une incoherence d'autant plus que l'autre table n'a pas de double dans cette colonne. 

In [48]:
#display(product.head())

xx = product[~product['LISTING_RECORD_CERTIFIED_THROUGH'].isnull()]
listingRecordDate = xx['LISTING_RECORD_CERTIFIED_THROUGH']
erreurListingRecordDate = []
for date in listingRecordDate:
    if not datee(date):
        erreurListingRecordDate.append(date)
print("Nombre incoherence dans LISTING_RECORD_CERTIFIED_THROUGH: ", len(erreurListingRecordDate))

ndcFlagY = product[product['NDC_EXCLUDE_FLAG'] != 'Y']
ndcFlagN = ndcFlagY[ndcFlagY['NDC_EXCLUDE_FLAG'] != 'N']
print("Nombre de rangee avec une valeur autre que Y ou N (ni E ou I) dans la colonne NDC_EXCLUDE_FLAG : ",len(ndcFlagN))

hh = product[~product['DEASCHEDULE'].isnull()]
print("Differentes possibilites pour DEASCHEDULE :",hh['DEASCHEDULE'].unique())

jj = product[~product['ACTIVE_INGRED_UNIT'].isnull()]
print("Differentes possibilites pour ACTIVE_INGRED_UNIT :",jj['ACTIVE_INGRED_UNIT'].unique())

kk = product[~product['ACTIVE_NUMERATOR_STRENGTH'].isnull()]
erreurActiveNumerator= [isinstance(x, numbers.Number) for x in kk]
print("Differentes possibilites pour ACTIVE_NUMERATOR_STRENGTH :",kk['ACTIVE_NUMERATOR_STRENGTH'].unique())
print("Il y a ",len(erreurActiveNumerator), "identifiants de ACTIVE_NUMERATOR_STRENGTH qui ne sont pas des nombres")

xx = product[~product['ENDMARKETINGDATE'].isnull()]
endMarketingDate = xx['ENDMARKETINGDATE']
erreurEndMarketingDate = []
for date in endMarketingDate:
    if not datee(date):
        erreurEndMarketingDate.append(date)
print("Nombre incoherence dans endMarketingDate: ", len(erreurEndMarketingDate))

yy = product[~product['STARTMARKETINGDATE'].isnull()]
startMarketingDate = yy['STARTMARKETINGDATE']
erreurStartdMarketingDate = []
for date in startMarketingDate:
    if not datee(date):
        erreurStartMarketingDate.append(date)
print("Nombre incoherence dans startMarketingDate: ", len(erreurStartdMarketingDate))

ll = product[~product['ROUTENAME'].isnull()]
print("Nombres de differentes possibilites pour ROUTENAME :",len(ll['ROUTENAME'].unique()))

zz = product[~product['DOSAGEFORMNAME'].isnull()]
print("Nombres de differentes possibilites pour DOSAGEFORMNAME :",len(zz['DOSAGEFORMNAME'].unique()))

cc = product[~product['PROPRIETARYNAMESUFFIX'].isnull()]
nameSuffix = []
for date in cc['PROPRIETARYNAMESUFFIX'].unique():
    if any(char.isdigit() for char in date):
           nameSuffix.append(date)
print("Nombres de differentes possibilites pour PROPRIETARYNAMESUFFIX :",len(nameSuffix))

vv = product[~product['PRODUCTNDC'].isnull()]
ndcErr = []
for ndc in vv['PRODUCTNDC'].unique():
    ndc = ndc.strip('-')
    if not any(char.isdigit() for char in ndc):
           ndcErr.append(ndc)
print("Nombres de differentes possibilites pour PRODUCTNDC:",len(ndcErr), ndcErr)

bb = product[~product['PRODUCTID'].isnull()]
print("Nombres de differentes possibilites pour PRODUCTID:",len(bb), len(bb['PRODUCTID'].unique()))

Nombre incoherence dans LISTING_RECORD_CERTIFIED_THROUGH:  0
Nombre de rangee avec une valeur autre que Y ou N (ni E ou I) dans la colonne NDC_EXCLUDE_FLAG :  0
Differentes possibilites pour DEASCHEDULE : ['CV' 'CIV' 'CIII' 'CII']
Differentes possibilites pour ACTIVE_INGRED_UNIT : ['mL/mL' 'mCi/mL' 'mg/.5mL' ... 'g/30mL; g/30mL; g/30mL; 1/30mL' '1/25mL'
 'mg/6g']
Differentes possibilites pour ACTIVE_NUMERATOR_STRENGTH : ['1' '51' '.75' ... '33.2' '415' '20; 25; 33']
Il y a  20 identifiants de ACTIVE_NUMERATOR_STRENGTH qui ne sont pas des nombres
Nombre incoherence dans endMarketingDate:  1
Nombre incoherence dans startMarketingDate:  0
Nombres de differentes possibilites pour ROUTENAME : 180
Nombres de differentes possibilites pour DOSAGEFORMNAME : 134
Nombres de differentes possibilites pour PROPRIETARYNAMESUFFIX : 1455
Nombres de differentes possibilites pour PRODUCTNDC: 9 ['OTC MONOGRAPH NOT FINAL', 'NDA', 'OTC MONOGRAPH FINAL', 'UNAPPROVED HOMEOPATHIC', 'UNAPPROVED MEDICAL GAS', 'U

Pour la colonne <b>DEASCHEDULE</b>, nous ne connaissons pas les valeurs des attributs il est donc difficile de savoir si il y a une incoherence.
<br><br>Pour la colonne <b>PHARM_CLASSES</b>, nous ne pouvons pas evaluer les incoherences en raison des valeurs de la colonne.
<br><br>Pour la colonne <b>ACTIVE_INGRED_UNIT</b>, il y a plusieurs valeurs, cependant en les analysant visuellement, il ne semble pas y avoir d'incoherences.
<br><br>Pour la colonne <b>ACTIVE_NUMERATOR_STRENGTH</b>, il y a des valeurs qui ne sont pas des nombres. Nous ne savons pas si ce sont des incoherences cependant, nous estimons pertinent de le souligner au cas ou cela pourrait poserait probleme plus tard.
<br><br>Pour la colonne <b>SUBSTANCENAME</b> nous ne pouvons pas savoir si les attributs qui sont dans les colonnes sont bons puisque nous n'avons pas de banque permettant de valider. Il en est de meme pour <b>APPLICATIONNUMBER LABELERNAME NONPROPRIETARYNAME PRODUCTTYPENAME PROPRIETARYNAME</b>, ce sont tous de objets, nous n'avons donc pas beaucoup de details quant a la composition des attributs.
<br><br>Pour la colonne <b>PROPRIETARYNAMESUFFIX</b> nous voyons qu'il y a 1455 elements que leur attribut contient un chiffre dans le suffix. Nous ne pouvons donc pas affirmer que de contenir un chiffre est un incoherence malgre que ce sont des suffix, le site indique qu'il n'y a pas de convention reconnue.
<br><br>Il est intéressant de voir que <b>PRODUCTNDC</b> ici est le premier attribut qui semble avoir des incohérences. En effet sur le site, il est indiqué que cet attribut est un chiffre séparé par un tiret. Or lorsque nous enlevons ces tirets, nous obtenons 9 sur quelque 90 000 attributs qui sont des mots. Nous avançons que ceux-ci sont des incohérences surtout que dans l'autre table nous n'observons pas cette situation.
<br><br> Nous pouvons voir qu'il n'y a pas de produit ayant le meme identifiant dans la colonne <b>PRODUCTID</b> car le nombre d'identifiants unique est le meme que le nombre d'identifiants.
<br><br>Pour ce qui est de <b>ROUTENAME MARKETINGCATEGORYNAME</b> nous n'avons pas trouver la liste des identifiants possible afin de confirmer que tous ceux present sont valide.


#### Conclusion 
Nous avons donc analysé toutes les colonnes afin de voir si il n'y avait pas d'erreurs ou d'incohérence dans les éléments de ces colonnes. Pour les dates nous avons vérifié qu'elles comportaient un mois entre 1 et 12 et un jour entre 1 et 31. Nous ne tenons pas compte de la variance du nombre de jours entre les mois. Lorsque la colonne comportait des objets qui par le nom nous permettait de croire que ce devrait être soi seulement composé de chiffres ou seulement composé de lettres, nous avons vérifié si c'était le cas et compté les nombre d'exceptions afin de vérifier si elles étaient répandues ou pas. Force est de constaté suite à notre analyse qu'il ne semble pas y avoir d'incohérence intra-attributs.

### Incohérences inter-Attributs

Lorsque l'on considère les différentes attributs pour les tables, la seule incohérence que nous pouvons étudier est la date, s'assurer que le début n'est pas après la fin. Il se pourrait qu'un produit ait une date de fin de marketing et pas de date de début, nous analyserons toutes ces possibilitées.

In [ ]:
for row in package:
    fin=package['ENDMARKETINGDATE']
    debut=package['STARTMARKETINGDATE']
    if str(debut)>= str(fin):
        print(debut,fin)
    if len(debut)==0 or len(fin)==0 :
        print("null")
        
for row in product:
    fin=product['ENDMARKETINGDATE']
    debut=product['STARTMARKETINGDATE']
    if str(debut)>= str(fin):
        print(debut,fin)
    if len(debut)==0 or len(fin)==0 :
        print("null")
        
aa = product[~product['PRODUCTNDC'].isnull()]
ss = product[~product['LABELERNAME'].isnull()]
print("Nombre de PRODUCTNDC et nombre de PRODUCTNDC unique:",len(aa), len(aa['PRODUCTNDC'].unique()))
print("Nombre de labelername et nombre de labelerName unique:",len(ss), len(ss['LABELERNAME'].unique()))

dd = product[~product['PROPRIETARYNAME'].isnull()]
ff = dd[~dd['PROPRIETARYNAME'].isna()]
print(len(ff))#['PROPRIETARYNAMESUFFIX'])

Il ne semble pas y avoir d'incoherences en ce qui concerne les dates dans les deux tables.
<br><br>Il y a une incoherence entre <b>ProductNDC</b> et <b>LabelerName</b>, il devrait y avoir le meme nombre d'attribut unique, or nous voyons qu'il y a une grande difference entre les deux.

In [ ]:
display(package.head())
display(product.head())